## Swiggy Restaurants Data Analysis

Notebook includes all answers to analysis question on swiggy.

Strting with all imports required.

In [1]:
import pandas as pd
import json
import os

Importing the `data.json` into a pandas dataframe and initializing common variables.

Before doing this, having insights of the structure of data.

Note: Please mount the drive and keep things in the same folder.

In [2]:
BASE_PATH = ('/content/drive/MyDrive/Colab Notebooks'
             '/GeeksforGeeks Hiring Data Science'
             '/Swiggy Restaurants Data Analysis')

with open(f'{BASE_PATH}/data.json', 'r') as json_file:
  json_data = json.loads(json_file.read())

Exploring data structured for cities and storing these in a variable for intial data conversion.

In [3]:
#Cities
cities = json_data.keys()

before doing same for subregions, as it is key under the city, we will make a method to make a `DataFrame` for each `city__subregion`. We make a partitioned data as a `dict` of dataframe will have all `restaurants`.

In [4]:
def load_clean_data(json_data: dict[str, object],
                    part_data: dict[str, pd.DataFrame],
                    city: str,
                    subregion: str | None = None) -> dict[str, pd.DataFrame]:
  if subregion is not None:
    key = f"{city}__{subregion}"
    jsn = json_data[city][subregion]['restaurants']
  else:
    key = f"{city}"
    jsn = json_data[city]['restaurants']
  if len(jsn) > 0:
    data = pd.DataFrame(jsn).transpose()
    cols  = ['name', 'rating', 'rating_count'] if 'rating_count' in data.columns else ['name', 'rating']
    if 'cost' in data.columns:
      cols.append('cost')
    if 'price' in data.columns:
      cols.append('price')
    data = (data[cols].dropna(axis=0, how='all'))
    data['rating'] = (data['rating']
                      .fillna('-1.0')
                      .str.replace('--', '0.0')
                      .str.replace('NA', '-1.0')
                      .astype(float))
    if 'cost' in cols:
      data['cost'] = (data['cost']
                      .fillna('0')
                      .str.replace('NA', '0')
                      .str.extract('(\d+)')
                      .astype(int))
    if 'price' in cols:
      data['price'] = (data['price']
                      .fillna('0')
                      .str.replace('NA', '0')
                      .str.extract('(\d+)')
                      .astype(int)) / 2
    if 'rating_count' in cols:
      data['rating_count'] = (data['rating_count']
                              .fillna('0')
                              .str.replace('NA', '0')
                              .str.replace('Too Few Ratings', '10')
                              .str.split('+', expand=True)[0]
                              .str.replace('K', '000')
                              .astype(int))
  else:
    data = None
  part_data[key] = data
  return part_data

In [5]:
part_data = {}
for city in cities:
  maybe_subcities = json_data[city].keys()
  if 'restaurants' in maybe_subcities:
    part_data = load_clean_data(json_data, part_data, city)
  else:
    subregions = set(maybe_subcities) - {'link'}
    for subregion in subregions:
      part_data = load_clean_data(json_data, part_data, city, subregion)
# part_data.keys()

We are ready with initial cleaning of data. Now lets start with answers to questoins.

### 1) How many cities (including subregions) where Swiggy is having its restaurants listed?

Here we will be counting our keys to `part_data` as it contains all combinations.

In [6]:
len(part_data.keys())

972

### 2) How many cities  (don't include subregions) where Swiggy is having their restaurants listed?

Our cities are already extracted so lets get the answers.

In [7]:
len(cities)

623

### 3) The Subregion of Delhi with the maximum number of restaurants listed on Swiggy?

Lets all the delhi subregion.

In [8]:
delhi_subregion = [key for key in part_data.keys() if key.startswith('Delhi__')]
delhi_subregion

['Delhi__South Extension',
 'Delhi__Laxmi Nagar',
 'Delhi__Kirti Nagar',
 'Delhi__Karol Bagh',
 'Delhi__Rohini',
 'Delhi__BBK_MayurVihar',
 'Delhi__Greater Kailash New',
 'Delhi__South Campus',
 'Delhi__Khan Market',
 'Delhi__Vasant Kunj',
 'Delhi__Lajpat Nagar',
 'Delhi__BBK_Chattarpur',
 'Delhi__Connaught Place',
 'Delhi__Punjabi Bagh',
 'Delhi__Ashok Vihar',
 'Delhi__Chattarpur',
 'Delhi__Janakpuri',
 'Delhi__Greater Noida',
 'Delhi__Uttam Nagar',
 'Delhi__Rajouri Garden',
 'Delhi__Old Delhi',
 'Delhi__Kaushambi',
 'Delhi__Noida Expressway',
 'Delhi__Jasola',
 'Delhi__Dwarka',
 'Delhi__GTB Nagar',
 'Delhi__Mayur Vihar',
 'Delhi__NSP',
 'Delhi__Indirapuram',
 'Delhi__Greater Kailash 2',
 'Delhi__Malviya Nagar',
 'Delhi__Tis Hazari',
 'Delhi__Dilshad Gardens',
 'Delhi__Sector 18',
 'Delhi__Golf Course']

Count the restaurants in each subreagion and summing up to seires.

In [9]:
(pd.Series({data_sub: len(part_data[data_sub])
            for data_sub in delhi_subregion
            if part_data[data_sub] is not None},
           index=delhi_subregion)
 .dropna()
 .sort_values()
 .tail(1).astype(int))

Delhi__Indirapuram    1279
dtype: int64

It is `Indirapuram` subregion where we have 1279 restaurants for `Delhi`.

### 4) Name the top 5 Most Expensive Cities in the Datasets.

For this lets have a quick check on the sum of cost in restaurant. Here we need to do a quick check on the columns for `cost` vs `price`.

The `cost` here represents the cost of dining, which we are looking for too.
The `price` says for two and already addressed in method above while initialization of data.

To get the top 5 most expencive cities, we need to normalize the data based on number of restaurants too.

In [10]:
def get_cost_normalized(df: pd.DataFrame) -> float:
  if 'cost' in df.columns and 'price' in df.columns:
    series = df['cost'] + df['price']
    return series.sum() / len(series)
  elif 'cost' in df.columns:
    return df['cost'].sum() / len(df['cost'])
  else:
    return df['price'].sum() / len(df['price'])


In [11]:
cities_cost = {}
for city in cities:
  city_data = part_data.get(city, None)
  if city_data is None:
    subregions = [value for key, value in part_data.items() if key.startswith(f'{city}__')]
    for df in subregions:
      subregion_price = get_cost_normalized(df) if df is not None else 0
      norm_factor = len(subregions) if df is not None else 1
    cities_cost[city] = cities_cost.get(city, 0) + (subregion_price / norm_factor)
  else:
    cities_cost[city] = get_cost_normalized(city_data) if city_data is not None else 0

In [12]:
pd.Series(cities_cost, index=cities_cost.keys()).sort_values().tail(5)

Mussoorie       460.000000
Gangtok         466.666667
North-goa       467.895726
South-goa       469.401024
Hinganghat    18962.437500
dtype: float64

We can see an exceptional hike in `Hinganghat` city which indicates an outlier in the given data, lets look for the problem in that.

In [13]:
for k,v in json_data['Hinganghat']['restaurants'].items():
  print(v['name'], v['cost'])

Govinda Restaurant ₹ 200
Atharav Bhojnalay ₹ 250
Hotel Royal ₹ 250
BABA BAKERS ₹ 300
The Foody Moody Cafe ₹ 250
HOTEL LAZEEZ ₹ 150
UPASANA MAGGI CENTER ₹ 150
Rocks Adda ₹ 200
Pet Pooja ₹ 100
Damino's pizza bite ₹ 300
KOHINOOR HOTEL ₹ 300350
U.F.C ₹ 150
Momos House ₹ 200
Rainbow Foods ₹ 200
Manoj Beakary ₹ 150
Caffe fresh ₹ 199


The problem lies in the outlier `KOHINOOR HOTEL` in the data. We are going to ignore this value, and can see what is the actual mean for this specific data, and replace the value accordingly.

In [14]:
(part_data['Hinganghat']['cost'].sum() - 300350) / (len(part_data['Hinganghat']['cost'] - 1))

190.5625

Considering `Hinganghat` value as `190`

In [15]:
top_5 = pd.Series(cities_cost, index=cities_cost.keys()).sort_values().tail(6)
top_5[:5].sort_values(ascending= False)

South-goa      469.401024
North-goa      467.895726
Gangtok        466.666667
Mussoorie      460.000000
Central-goa    421.618812
dtype: float64

### 5) List out the top 5 Restaurants with Maximum & minimum ratings throughout the dataset.

Here we need a full dataset operation so taking full frame with required columns only.

in our current question we need 2 factors:
- `ratings`
- `rating_counts`

We need rating for describing the overall raiting and we will use `rating_counts` for normalizing the restaurant rates.


In [16]:
combined_df = pd.DataFrame(columns=['name', 'rating', 'rating_count', 'place'])
for city in cities:
  city_data = part_data.get(city, None)
  if city_data is None:
    subregions = [(key, value) for key, value in part_data.items() if key.startswith(f'{city}__')]
    for subregion in subregions:
      if subregion[1] is not None and 'rating_count' in subregion[1].columns:
        subregion[1]['place'] = [subregion[0]] * len(subregion[1])
        combined_df = pd.concat([combined_df, subregion[1][['name', 'rating', 'rating_count', 'place']]], axis = 0)
  else:
    if 'rating_count' in city_data.columns:
      city_data['place'] = [city] * len(city_data)
      combined_df = pd.concat([combined_df, city_data[['name', 'rating', 'rating_count', 'place']]], axis = 0)
combined_df.head()

,name,rating,rating_count,place
567335,AB FOODS POINT,0.0,10,Abohar
531342,Janta Sweet House,4.4,50,Abohar
158203,theka coffee desi,3.8,100,Abohar
187912,Singh Hut,3.7,20,Abohar
543530,GRILL MASTERS,0.0,10,Abohar


In [17]:
len(combined_df)

157288

Implementing IMDB rating formula for calculating weighted weights.

`weighted rating (WR)=(v/(v+m))R+(m/(v+m))C`

where:
- R = average for the restaurant (mean) = (rating)
- v = number of ratings for the restraunt = (rating_count)
- m = minimum rating_count required to be considered
- C = the mean vote across the whole dataset.

In [18]:
mask = combined_df['rating']
r = combined_df['rating']
v = combined_df['rating_count']
m = 10 # for our dataset considering too few as 10
c = r.mean()

combined_df['weighted rating'] = ((v/(v+m))*r) + ((m/(v+m))*c)

Top 5 restaurants rated

In [19]:
combined_df[['weighted rating','name', 'place']].sort_values('weighted rating').tail(5)

,weighted rating,name,place
37107,4.737636,Natural Ice Cream,Kolkata__Central Kolkata
326540,4.737636,Natural Ice Cream,Kolkata__South Kolkata
363536,4.835675,Kwality Walls Frozen Dessert and Ice Cream Shop,Ludhiana__Dugri
309376,4.835675,Kwality Walls Frozen Dessert and Ice Cream Shop,Delhi__Jasola
467010,4.835675,SUNSHINE SHAKE,Rohtak


Last 5 Restaurants

In [20]:
combined_df[['weighted rating','name', 'place']].sort_values('weighted rating').head(5)

,weighted rating,name,place
567335,0.809709,AB FOODS POINT,Abohar
422692,0.809709,She Tray,Kochi__Panampilly Nagar
235190,0.809709,Lantern,Kochi__Panampilly Nagar
566555,0.809709,KALAVARA,Kochi__Panampilly Nagar
380607,0.809709,RAO'S FOOD COURT,Kochi__Panampilly Nagar


### 6) Name of top 5 cities with the highest number of restaurants listed.

listing top five cities where these resturants are.

In [21]:
(pd.DataFrame({'city': [city.split('__')[0] for city in part_data.keys()
                        if part_data[city] is not None],
               'number': [len(part_data[data_sub]) for data_sub in part_data.keys()
                          if part_data[data_sub] is not None]})
).groupby('city').sum().sort_values('number').tail(5).sort_values('number', ascending=False)

,number
city,
Bangalore,15305
Delhi,14085
Pune,12446
Hyderabad,11892
Chennai,9957


### 7) Top 10 cities as per the number of restaurants listed?

In [22]:
(pd.DataFrame({'city': [city.split('__')[0] for city in part_data.keys()
                        if part_data[city] is not None],
               'number': [len(part_data[data_sub]) for data_sub in part_data.keys()
                          if part_data[data_sub] is not None]})
).groupby('city').sum().sort_values('number').tail(10).sort_values('number', ascending=False)

,number
city,
Bangalore,15305
Delhi,14085
Pune,12446
Hyderabad,11892
Chennai,9957
Kolkata,8287
Mumbai,6108
Jaipur,5540
Gurgaon,3601


### 8) Name the top 5 Most Popular Restaurants in Pune.

Lets list top 5 restaurants in pune. Presuming the formula of IMDB rating we can follow the ratings of most popular restaurant.

Lets work with `combined_df` and filter it for `Pune`

In [23]:
(combined_df[
    combined_df['place']
    .str
    .split('__', expand=True)[0] == 'Pune'
    ][['rating_count','name', 'weighted rating']]
 .sort_values(['rating_count','weighted rating'])
 .tail(5))

,rating_count,name,weighted rating
135737,1000,Sandwich Express,4.372469
238148,1000,Theobroma,4.471479
42906,1000,Sheetal Restaurant,4.471479
12450,1000,Irani Cafe,4.471479
180419,1000,Kalika Dairy & Sweets,4.471479


### 9) Which SubRegion in Delhi is having the least expensive restaurant in terms of cost?

Lets use `delhi_subregion` for getting a subset of data for all delhi subgroups and then query on costs to get least costliest subgroup.

In [24]:
pd.Series([part_data[subregion]['cost'].sum()/len(part_data[subregion])
           for subregion in delhi_subregion
           if part_data.get(subregion, None) is not None],
          index = [subregion.split('__')[1] for subregion in delhi_subregion
                       if part_data.get(subregion, None) is not None]
).sort_values().head(1)

Sector 18    236.666667
dtype: float64

Seems the avg cost of dining is lowest in `Sector 18` subregion of Delhi, i.e `236.67`

### 10) Top 5 most popular restaurant chains in India?

Lets follow the `combined_df` for comparing popularity, and then group by `name` to take out **chains** allover dataset and then by `sort('rating_count', 'weighted rating')`, lets find what is the most popular chan.



In [25]:
chn = []
chains = combined_df.groupby('name')['name'].count()
for cnt, name in zip(chains, chains.index):
  if cnt > 1:
    chn.append(name)

(combined_df[combined_df['name'].isin(chn)]
 .groupby('name')[['rating_count', 'weighted rating']]
 .max()
 .sort_values(['rating_count', 'weighted rating'])
 .tail(5))

,rating_count,weighted rating
name,,
Eaters Stop,10000,3.997622
Mehfil,10000,3.997622
Meridian Restaurant,10000,3.997622
Shah Ghouse Cafe & Restaurant,10000,4.075440
Grand Hotel,10000,4.097522


### 11) Which restaurant in Pune has the most number of people visiting?



In [26]:
(combined_df[
    combined_df['place']
    .str
    .split('__', expand=True)[0] == 'Pune'
    ][['rating_count','name', 'weighted rating']]
 .sort_values(['rating_count','weighted rating'])
 .tail(1))

,rating_count,name,weighted rating
180419,1000,Kalika Dairy & Sweets,4.471479


The most visited restaurant in `Pune` is `Kalika Dairy & Sweets`. Because of 2 factors:
- The rating counts here is above 1000 users.
- The rating to this is highest among all in `Pune`


12) Top 10 Restaurants with Maximum Ratings in Bangalore

for checking in bangalore, we will follow the same procedure i.e `weighted rating`. It makes more sence to see the normalized for of ratings and not the actual rating.

In [27]:
(combined_df[
    combined_df['place']
    .str
    .split('__', expand=True)[0] == 'Bangalore'
    ][['rating_count','name', 'weighted rating']]
 .sort_values('weighted rating')
 .tail(10)
 .sort_values('weighted rating',ascending=False))

,rating_count,name,weighted rating
13969,1000,Corner House Ice Cream,4.669499
101999,1000,NIC Natural Ice Creams,4.669499
29674,500,Natural Ice Cream,4.639596
29668,500,Natural Ice Cream,4.639596
93904,500,NIC Natural Ice Creams,4.639596
29672,500,Natural Ice Cream,4.639596
248725,500,Natural Ice Cream,4.639596
52431,1000,The Filter Coffee,4.570489
8174,1000,Corner House Ice Cream,4.570489
444178,1000,Magnolia Bakery,4.570489


### 13) Top 10 Restaurant in Patna w.r.t rating


In [28]:
(combined_df[
    combined_df['place']
    .str
    .split('__', expand=True)[0] == 'Patna'
    ][['rating_count','name', 'weighted rating']]
 .sort_values('weighted rating')
 .tail(10)
 .sort_values('weighted rating',ascending=False))

,rating_count,name,weighted rating
290590,1000,Kwality Walls Frozen Dessert and Ice Cream Shop,4.570489
251514,500,Bikaner Sweets And Pastry Shop,4.24744
245871,1000,HARILAL ( S.K. Puri ),4.17445
550814,50,Slurpy Shakes,4.103236
259295,1000,Bikaner Elite ( Rukanpura ),4.07544
246443,1000,HARILAL (Kankarbagh),4.07544
289244,100,HARILAL (Gandhi Maidan),4.056311
125716,100,Mr. Bakers,4.056311
424138,100,FreshBox,4.056311
254859,500,Biryani By Kilo,4.051361
